In [116]:
import json
from collections import OrderedDict
import types

_FUNC = (types.FunctionType, types.MethodType)

class OptionMeta(type):
    def __new__(mcs, name, bases, kw):
        fields = []
        fields.extend(k for k in kw if not isinstance(kw[k], _FUNC) and k not in {"__qualname__", "__module__"})
        kw['_fields_'] = fields
        return type.__new__(mcs, name, bases, kw)
        
    def __prepare__(self, bases):
        return OrderedDict()


class OptionCategory(metaclass=OptionMeta):
    def __iter__(self):
        cd = self.__class__.__dict__
        sd = self.__dict__
        rv = []
        for f in self._fields_:
            if f[0] == "_" or f[-1] == "_": continue
            try:
                v = sd[f]
            except KeyError:
                v = cd[f]
            rv.append((f, v))
        return iter(rv)
    
    def __repr__(self):
        buf = []
        for k, v in self:
            if isinstance(v, OptionCategory):
                v = "<More Options...>"
            buf.append("%s: %s" %(k,v))
        return "\n".join(buf)
    
    def __init__(self):
        self.__dict__.update(self.__class__.__dict__)
    
    def __setattr__(self, k, v):
        if k not in self.__dict__:
            raise AttributeError(k)
        elif k == "_fields_":
            raise AttributeError("Can't set _fields_ property")
        object.__setattr__(self, k, v)
        
    def asdict(self):
        rv = OrderedDict()
        for k in self._fields_:
            v = getattr(self, k)
            if isinstance(v, OptionCategory):
                v = v.asdict()
            rv[k] = v
        return rv
    
    def jsonify(self, **kw):
        kw['indent'] = kw.get('indent') or 4
        return json.dumps(self.asdict(), **kw)
    
    def pretty_string(self, expand=False, indent=0):
        buf = []
        ind = indent * "    "
        for k, v in self:
            if isinstance(v, OptionCategory):
                if expand:
                    s = "\n" + v.pretty_string(expand, indent+1)
                else:
                    s = "<More Options...>"
            else:
                s = str(v)
            buf.append("%s%s: %s" %(ind, k,s))
        return "\n".join(buf)

    def __repr__(self):
        return self.pretty_string(False, 0)

In [117]:
import json

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    gamma = 0
    deadband = 0
    man_request = 0
    mode = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72
    xscale_factor = 3600

class SimOps(OptionCategory):
    
    co2_pid = PIDOps()
    co2_pid.p = -200
    co2_pid.i = 2
    co2_pid.d = 0
    co2_pid.amax = 100
    co2_pid.amin = 0
    co2_pid.beta = 1
    co2_pid.linearity = 1
    co2_pid.alpha = -1
    
    base_pid = PIDOps()
    base_pid.p = 20
    base_pid.i = 2
    base_pid.d = 0
    base_pid.amax = 100
    base_pid.amin = 0
    base_pid.beta = 0
    base_pid.linearity = 1
    base_pid.alpha = -1

    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 10
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    plots.xscale_factor = 3600
    
    delay = 0
    end = 10000
    initial_actual_cno = (1,1,1)
    initial_request_cno = (0.07, 0, 0)
    initial_request_base = 0
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    dc = 0
    d2c = 0
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80
    time_unit = 3600
    max_iters = 3 * 86400
    bicarb = 3.7  # g/L
    temp = 37
    base_bicarb = 0.5  # mol/L
    hcalc_method = "h+"
simops = SimOps()

In [118]:
print(simops.pretty_string(True,0))

co2_pid: 
    p: -200
    i: 2
    d: 0
    amax: 100
    amin: 0
    alpha: -1
    beta: 1
    linearity: 1
    gamma: 0
    deadband: 0
    man_request: 0
    mode: 0
base_pid: 
    p: 20
    i: 2
    d: 0
    amax: 100
    amin: 0
    alpha: -1
    beta: 0
    linearity: 1
    gamma: 0
    deadband: 0
    man_request: 0
    mode: 0
mfcs: 
    co2_max: 1
    o2_max: 10
    n2_max: 10
    air_max: 10
plots: 
    xscale: auto
    xmin: 0
    xmax: 72
    xscale_factor: 3600
delay: 0
end: 10000
initial_actual_cno: (1, 1, 1)
initial_request_cno: (0.07, 0, 0)
initial_request_base: 0
initial_pv: 90
set_point: 40
set_point_deadband: 1
k_mult: 1.1
k: None
c: None
dc: 0
d2c: 0
mode: m2a
main_gas: 1.0
reactor_size: 80
reactor_volume: 55.0
time_unit: 3600
max_iters: 259200
bicarb: 3.7
temp: 37
base_bicarb: 0.5
hcalc_method: h+


In [85]:
print(simops.jsonify())

{
    "co2_pid": {
        "p": -200,
        "i": 2,
        "d": 0,
        "amax": 100,
        "amin": 0,
        "alpha": -1,
        "beta": 1,
        "linearity": 1,
        "gamma": 0,
        "deadband": 0,
        "man_request": 0,
        "mode": 0
    },
    "base_pid": {
        "p": 20,
        "i": 2,
        "d": 0,
        "amax": 100,
        "amin": 0,
        "alpha": -1,
        "beta": 0,
        "linearity": 1,
        "gamma": 0,
        "deadband": 0,
        "man_request": 0,
        "mode": 0
    },
    "mfcs": {
        "co2_max": 1,
        "o2_max": 10,
        "n2_max": 10,
        "air_max": 10
    },
    "plots": {
        "xscale": "auto",
        "xmin": 0,
        "xmax": 72,
        "xscale_factor": 3600
    },
    "delay": 0,
    "end": 10000,
    "initial_actual_cno": [
        1,
        1,
        1
    ],
    "initial_request_cno": [
        0.07,
        0,
        0
    ],
    "initial_request_base": 0,
    "initial_pv": 90,
    "set_point":